# Iteration and Stability

**ENGSCI233: Computational Techniques and Computer Systems** 

*Department of Engineering Science, University of Auckland*

# 0 What's in this Notebook?

Differential equations describe how physical systems change over time. If we want to predict those changes into the future, we need to solve the equations. Most can't be solved analytically.

The next best option is a numerical method. You have already seen Euler's method - its not very accurate, although the philosophy is spot on. Estimate the derivative, use it to take a jump forward, re-evaluate the derivative. Here, we'll explore some more complex methods, and how to measure their performance.

You need to know:
- How to use jumps back and forth - predictor and corrector steps - when implementing an improved version of Euler's method. This concept generalises to higher order Runge-Kutta methods.
- The accuracy of these methods depends on how large a step we take. We can use a convergence test to decide how small those steps need to be for a decent result.
- Sometimes these methods are unstable. That is both a drawback of the method and but also depends on the difficulty of the problem it has been applied to (stiffness).

Note for user: **equations format poorly** in this notebook. If they are running off the screen "re-run" the cell containing the equation (click on the cell, hit Enter, then Ctrl+Enter).

In [ ]:
# imports and environment: this cell must be executed before any other in the notebook
%matplotlib inline
from matplotlib import pyplot as plt

## 1 ODE solution methods

<mark>***Different methods to approximately solve Ordinary Differential Equations***</mark>

Consider the initial value problem $\frac{dy}{dx} = (1 + xy)^{2}, y(0) = 1$.  This equation is **non-separable** and **nonlinear** which makes it difficult to solve.  An approximate solution can still be generated using a numerical method.

In this module, we will consider **first-order** ODEs of the form $\frac{dy}{dx} = f(x,y)$.  Higher-order ODEs can be written as a system of first-order ODEs so this approach can be applied to almost any ODE. Furthermore, we will restrict our interest to **initial value problems**, i.e., cases where information about $y$ (and its derivatives) is known at some starting **reference point** $x^{(0)}$.

The numerical methods we will consider are all **step-by-step** methods.  This means that we will start from the **known solution** $y^{(0)}$ at $x^{(0)}$ and proceed **stepwise**.  First, we take a step of length $h$ and **estimate a solution** $y^{(1)}$ at $x^{(1)} = x^{(0)} + h$.  Then, we use the value $y^{(1)}$ as a **pseudo-initial value** to estimate a solution $y^{(2)}$ at $x^{(2)} = x^{(1)} + h = x^{(0)} + 2h$.  Proceeding in this manner we will be able to construct an approximate solution for $y$ over a specified range of $x$.

### 1.1 Euler method

All the methods in this module match the derivatives in a **Taylor's expansion** over a finite region to some order. The omission of **higher-order** Taylor's series terms in the approximation means that each method will have an associated **error**.  The **Euler method** is the simplest in that it only approximates the first derivative of the function. It does this using a first-order **finite difference** approximation to the derivative.

The finite difference approximation to the derivative is:

\begin{equation}
  \frac{dy}{dx} \thickapprox \dfrac{\Delta y}{\Delta x} = \dfrac{y^{(k+1)} - y^{(k)}}{h} = f\left(x^{(k)},y^{(k)}\right)
\end{equation}

This can be rearranged to give

\begin{equation}
  y^{(k+1)} = y^{(k)} + h\,f\left(x^{(k)},y^{(k)}\right) 
\end{equation}

where $f$ is a known function for the derivative.

***Execute the cell below for a demonstration of the Euler method for the ODE $dy/dx = (1+xy)^2$***

In [ ]:
# import functions
from ipywidgets import interact
from iteration233 import plot_euler_elements

def euler_figure(steps=0.5, h = 0.1):
    # create figure
    f,(ax1, ax2) = plt.subplots(1,2)
    f.set_size_inches(12,5)
    
    # display plot
    plot_euler_elements(ax1, ax2, steps, h)

# run the interactive figure environment
interact(euler_figure, steps = (0.5,10,0.5), h = (0.02,0.2,0.02));

***How does error in the approximation change with the step-size, $h$? Describe any trade-offs.***

> <mark>*~ your answer here ~*</mark>

***What type of error is introduced by truncating the Taylor series to a finite number of terms?***

> <mark>*~ your answer here ~*</mark>


The exercise below attempts to solve the ODE

\begin{equation}
\frac{dy}{dx} = \sin\left(a\sin(x)\sqrt{x}+\frac{\cos(bx)}{x+1}\right)
\end{equation}

However, we should be mindful of the tradeoff between **efficiency and accuracy** (or effort and error).

***Execute the cell below. Experiment with the input boxes.***

***In general, how many steps are required to get the error below 5%?***

In [ ]:
# import functions
from ipywidgets import fixed
from iteration233 import plot_tradeoffs_elements, tradeoffs_setup

# function parameters
p = [8, 8.5]
def tradeoffs_figure(steps, predict_value, p):
    # create figure
    f,ax = plt.subplots(1,1)
    f.set_size_inches([16, 6])
    
    # display plot
    plot_tradeoffs_elements(ax, steps, predict_value, p)

# run the interactive figure environment
box1,box2 = tradeoffs_setup()
interact(tradeoffs_figure, steps = box1, predict_value = box2, p = fixed(p));

### 1.2 Improved Euler method

The Euler method uses the slope (i.e., the value of $y'$) at $y^{(k)}$ to predict $y^{(k+1)}$.  If the slope is **changing significantly** over the step length then the Euler method will give an **inaccurate** solution.  An improved method can be devised by **averaging** the slopes at $y^{(k)}$ and $y^{(k+1)}$.

The slope at $y^{(k)}$ is $f\left(x^{(k)}, y^{(k)}\right)$. A **prediction** of the solution at $x^{(k+1)}$ is $y^{(k+1)}_p=y^{(k)}+h\,f\left(x^{(k)}, y^{(k)}\right)$. The slope at $y^{(k+1)}$ is $f\left(x^{(k+1)}, y^{(k+1)}_p\right)$. 

Using the average slope in the previous definition of the Euler method yields the **Improved Euler method**:

\begin{equation}
\begin{split}
y^{(k+1)} &= y^{(k)} + h \times \text{average slope} \\
&= y^{(k)} + \dfrac{h}{2} \left[f\left(x^{(k)},y^{(k)}\right) + f\left(x^{(k)}+h,y^{(k)} + h\,f\left(x^{(k)},y^{(k)}\right)\right)\right].    
\end{split}
\end{equation}

The Improved Euler Method is an example of a **predictor-corrector** method. First, the value $y^{(k+1)}_p$ is predicted using an Euler step. This allows extra slope information to be determined which can be used to make a corrected estimate of $y^{(k+1)}$.

The Improved Euler Method is a **second-order** method, i.e., truncation error $\sim\mathcal{O}(h^{3})$. The method is computationally **more expensive** than the Euler Method but it provides **more accurate** solutions.

***Execute the cell below for a demonstration of the Improved Euler method for the ODE $dy/dx = (1+xy)^2$***

In [ ]:
# import functions
from iteration233 import plot_improved_elements

def improved_euler_figure(steps=6.25, compare_euler=False):
    # create figure
    f,ax = plt.subplots(1,1)
    f.set_size_inches([10,4])
    
    # display plot
    plot_improved_elements(ax, steps, compare_euler)

# run the interactive figure environment
interact(improved_euler_figure, steps = (0.25,9,0.25), compare_euler = False);

### 1.3 Runge-Kutta methods

Could you design a better method than improved Euler, that builds complexity in the same manner Euler -> improved Euler? One thing we could do is evaluate a derivative at the mid-point of the step and use that information to compute a **weighted** derivative, e.g.,

\begin{equation}
\begin{split}
f_0 & = f\left(x^{(k)}, y^{(k)}\right) \\
f_1 & = f\left(x^{(k)}+\frac{h}{2}, y^{(k)}+\frac{h}{2}f_0\right) \\
f_2 & = f\left(x^{(k)}+h, y^{(k)}-hf_0+2hf_1\right) \\
y^{(k+1)} &= y^{(k)} + \dfrac{h}{6} \left(f_0 + 4f_1 + f_2\right).    
\end{split}
\end{equation}

This is a third order scheme with truncation error $\sim\mathcal{O}(h^{4})$.

The schemes get more complicated, although its diminishing returns beyond 4th order. However, they all fall under a broad category called Runge-Kutta methods. A particularly popular implementation is called **RK45**.

## 2 Accuracy

<mark>***Measuring how close a numerical solution is to the true solution.***</mark>

Numerical computation provides a number of opportunities for **error** to creep in. Improvements to algorithms can **minimize** this error - usually at the **cost** of execution time - but can **never eliminate** it entirely. It is therefore useful to **estimate** the error that accompanies a particular solution method.

### 2.1 Euler method

If $y(x^{(k)}+h)$ is the exact value at $x^{(k)}+h$ then, by Taylor's series:

\begin{equation}
  \begin{split}
    y\left(x^{(k)}+h\right) &= y\left(x^{(k)}\right) + h\,\frac{dy}{dx}\left(x^{(k)}\right) + \dfrac{h^{2}}{2!} \frac{d^2y}{dx^2}\left(x^{(k)}\right) + 
    \dfrac{h^{3}}{3!} \frac{d^3y}{dx^3}\left(x^{(k)}\right)+ \ldots \\
    &= \text{Euler's Method estimate} + \mathcal{O}(h^{2})
  \end{split}
\end{equation}

and thus the **truncation error** at each step involves terms that are proportional to $h^{2}$ or higher powers.

If we halve the step length then the error per step is proportional to $\left(\dfrac{h}{2}\right)^{2} = \dfrac{h^{2}}{4}$. However, to get to the same point we must do twice as many steps. Thus the total error is $E \varpropto 2 \left(\dfrac{h^{2}}{4}\right)= \dfrac{h^{2}}{2}$. Thus, by **halving the step length we halve the error** (and by doubling the step length we double the error). Because error for Euler's method is proportional to step length, it is termed a **first-order method**. In general, an $n$-th order method will have truncation error of $\mathcal{O}(h^{n+1})$.

### 2.1 Improved Euler method

The Improved Euler method is **second order accurate**, i.e., truncation error $\sim \mathcal{O}(h^{3})$, but this improvement over the Euler method comes at the cost of **greater computational expense**.

From the formulation of the Improved Euler method, the derivative estimated at the **corrector step** can be expanded (applying a first order Taylor expansion, first for $x$, then for $y$)

\begin{equation}
\begin{split}
f\left(x^{(k)}+h,y^{(k)} + h\,f\left(x^{(k)},y^{(k)}\right)\right) &=&\,f\left(x^{(k)},y^{(k)} + h\,f\left(x^{(k)},y^{(k)}\right)\right) + h \frac{\partial f}{\partial x}\left(x^{(k)},y^{(k)} + h\,f\left(x^{(k)},y^{(k)}\right)\right) \\
&=&\,f\left(x^{(k)},y^{(k)}\right) +  h\,f\left(x^{(k)},y^{(k)}\right)\,\frac{\partial f}{\partial y}\left(x^{(k)},y^{(k)}\right) + h \left(\frac{\partial f}{\partial x}\left(x^{(k)},y^{(k)}\right) + h\,f\left(x^{(k)},y^{(k)}\right)\,\frac{\partial^2 f}{\partial x\partial y}\left(x^{(k)},y^{(k)}\right)\right)  \\
&=&\,f\left(x^{(k)},y^{(k)}\right) + h \frac{df}{dx}\left(x^{(k)}, y^{(k)}\right) + h^2\,f\left(x^{(k)},y^{(k)}\right)\,\frac{\partial^2 f}{\partial x\partial y}\left(x^{(k)},y^{(k)}\right),
\end{split}
\end{equation}

where the total derivative is expressed $\frac{df}{dx}=\frac{\partial f}{\partial x}+\frac{\partial y}{\partial x}\frac{\partial f}{\partial y}$. Substituting this expression into the Improved Euler formula we get

\begin{equation}
\begin{split}
y^{(k+1)} &=&\,y^{(k)} + h\,f\left(x^{(k)},y^{(k)}\right) + \frac{h^2}{2}\frac{df}{dx}\left(x^{(k)}, y^{(k)}\right) + \mathcal{O}(h^3)\\
&=&\,\text{Improved Euler method estimate} + \mathcal{O}(h^3).
\end{split}
\end{equation}


### 2.3 Runge-Kutta methods

We won't bother with a full derivation of truncation error for the higher order methods. It is more important to appreciate the concept: if a method is $n^{th}$-order, then it has truncation error $\sim\mathcal{O}(h^{n+1})$. Halving the step size means that error goes from $h^{n+1} \rightarrow h^{n+1}/2^{n+1}$, but we need to take twice the number of steps, so the total error is $h^{n+1}/2^n$. So the accuracy is improved, but that comes at a cost of having to evaluate more derivatives at ever more midpoints.

***Execute the cell below to compare accuracy of the different schemes for different step sizes.***

In [ ]:
# import functions
from iteration233 import plot_accuracy_elements

# function parameters
def accuracy_figure(more_steps=0):
    # create figure
    f,(ax,ax2) = plt.subplots(1,2)
    f.set_size_inches([12,4])

    # display plot
    plot_accuracy_elements(ax, ax2, more_steps)

# run the interactive figure environment
interact(accuracy_figure, more_steps = (0,15,1));

***What are the slopes of the curves on the RIGHTHAND plot?***

> <mark>*~ your answer here ~*</mark>

***Which method is the most accurate?***

> <mark>*~ your answer here ~*</mark>

## 3 Stability

<mark>***Eliminating the wiggles and wobbles in the numerical solution.***</mark>

### 3.1 Euler method

Consider the simple ODE $y' = ay$, which we know has an exponential solution. For the case that $a<0$, the solution should decay to zero.

The Euler update step is 

\begin{equation}
\begin{split}
y^{(k+1)} &=&\,y^{(k)} + h\,a\,y^{(k)} \\
&=&\,\left(1+ha\right)y^{(k)} \\
&=&\,\left(1+ha\right)^k y^{(0)}.
\end{split}
\end{equation}

If $|1+ha|>1$ then $y^{(k)}$ will keep increasing in value rather than decaying to 0. This defines a **stability criterion** for Euler's method applied to this problem: $-1 < 1+ha < 1 \Rightarrow 1 + ha > -1 \Rightarrow ha > -2 \Rightarrow h < - \dfrac{2}{a}$.  Therefore the stability criterion for the Euler method to give stable solutions for this problem is $h < -\dfrac{2}{a}$.

### 3.2 Improved Euler method

The Improved Euler method is a second-order accurate method and therefore we would expect it to provide **more accurate** solutions than the Euler method for the same choice of step size. However, the stability of the method is **not necessarily improved** as the solution still relies on the Euler estimate of $y_p^{(k+1)}$. 

The Improved Euler update step is 

\begin{equation}
\begin{split}
y^{(k+1)} &=&\,y^{(k)} + \frac{h}{2}\,\left(a\,y^{(k)}+a(1+ha)y^{(k)}\right) \\
&=&\,\left(1+ha+\frac{h^2}{2}a^2\right)y^{(k)} \\
&=&\,\left(1+ha+\frac{h^2}{2}a^2\right)^k y^{(0)}.
\end{split}
\end{equation}

For $a<0$ we require $y^{(k)}$ to decrease and not oscillate in sign, which implies $0 < 1 + ha + \tfrac{h^2}{2} a^2 < 1$. Thus, if 

\begin{equation}
1 + ha + \tfrac{h^2}{2} a^2 < 1, \quad\Rightarrow\quad ha ( 1 + \tfrac{h}{2} a ) < 0 \quad\Rightarrow\quad h < -\tfrac{2}{a},
\end{equation}

which is the **same criterion** for the Euler method. 

Similarly, if 

\begin{equation}
1 + ha + \tfrac{h^2}{2} a^2 > 0 \quad\Rightarrow\quad 2 + 2ha + h^2 a^2 > 0 \quad\Rightarrow\quad ( 1 + ha ) ^2 > -1
\end{equation}

which is **always true** since the left-hand-side is positive and the right-hand-side is negative for any choice of $h$. Therefore the criterion for the Improved Euler method to give **stable, non-oscillating** solutions for this problem is $h < -\tfrac{2}{a}$.

***Execute the cell below to see how stability depends on method type and step size***

In [ ]:
# import function
from iteration233 import plot_stability_elements

# function parameters
def stability_figure(steps=28, method='Euler'):
    # create figure
    f,ax = plt.subplots(1,1)
    f.set_size_inches([10,4])

    # display plot
    plot_stability_elements(ax, steps, method)

# run the interactive figure environment
interact(stability_figure, steps = (3,15,1), method = ['Euler','Improved Euler','RK45']);

***Set STEPS to 15. Which method is the most accurate?***

> <mark>*~ your answer here ~*</mark>

***For the equation above, $a$=-10. For what step size does the Euler method exhibit oscillation?***

> <mark>*~ your answer here ~*</mark>

***Set STEPS to 6. Which method is the most accurate?***

> <mark>*~ your answer here ~*</mark>

***For what step size does the Euler method exhibit instability?***

> <mark>*~ your answer here ~*</mark>

### 3.1 Stiffness

The example above illustrates a quality of some differential equations called **stiffness**. While the definition of stiffness is a little nebulous, the following characteristics can be used to diagnose stiff solutions:
- Method stability is **constrained** by step size.
- Derivatives depend strongly on, or are large compared to, the solution.
- Unnecessarily **expensive** with explicit methods like Euler, improved Euler, RK45. Performs better with **implicit** methods (next year).

Stiffness is a **property of the solution** whereas stability is a property of the **solution method**. Consider the example below

\begin{equation}
\frac{dy}{dx}=a(x)y,\quad\quad\begin{Bmatrix} & 0.1 \quad & x < 20 \\ a(x) = & 1\quad & 20\leq x < 40 \\ & 0.1 \quad & x \geq 40 \end{Bmatrix},\quad\quad y(0)=1
\end{equation}

***Execute the cell below for a demonstration of solution stiffness.***

In [ ]:
# import functions
from iteration233 import plot_stiffness_elements

# function parameters
def stiffness_figure(step_size=0.5, log_yscale=False):
    # create figure
    f,(ax,ax2) = plt.subplots(1,2)
    f.set_size_inches([12,5])

    # display plot
    plot_stiffness_elements(ax, ax2, step_size)
    if log_yscale: ax.set_yscale('symlog',linthreshy=1.e-5)

# run the interactive figure environment
interact(stiffness_figure, step_size = (0.75,2.5,0.25), log_yscale=False);

***Set the STEP_SIZE to 0.75. Does the Euler method to a reasonable job solving this problem?*** 

> <mark>*~ your answer here ~*</mark>

***Set the STEP_SIZE to 1.75. Where have solution oscillations started to appear? Use the righthand plot to explain the appearance of solution oscillations in terms of the step size and value of $a$.*** 

> <mark>*~ your answer here ~*</mark>

***Set the STEP_SIZE to 2.75. Where is the solution exhibiting stiffness? Use the righthand plot to explain how ONLY PART of the solution can be stiff. Explain the stiffness in terms of the step size and value of $a$.*** 

> <mark>*~ your answer here ~*</mark>


## 4 Convergence

<mark>***Determining when a numerical solution is close enough to the true solution.***</mark>

Most ODEs do not have an **analytical solution**. How should we determine that a solution is **sufficiently accurate** when the true solution can never be known? One way is to solve the ODE for a range of step sizes, $h$, and look for **convergent behaviour**. 

For example, take the ODE in Section 3 with $a=10$, $y(0) = 1$ and say we wish to estimate $y(0.1)$

***Execute the cell below for a demonstration of ODE convergence.***

In [ ]:
# import functions
from iteration233 import plot_convergence_elements

# function parameters
def convergence_figure(more_steps=0, zoom=False):
    # create figure
    f,(ax,ax2) = plt.subplots(1,2)
    f.set_size_inches([12,4])

    # display plot
    plot_convergence_elements(ax, ax2, more_steps, zoom)

# run the interactive figure environment
interact(convergence_figure, more_steps = (0,15,1), zoom = False);

***Set MORE_STEPS to 3. How has the ODE solution at $x=0.2$ changed by taking smaller steps?*** 

> <mark>*~ your answer here ~*</mark>

***What does the horizontal axis on the righthand plot represent?*** 

> <mark>*~ your answer here ~*</mark>

***Set MORE_STEPS to 15. What value is the solution converging toward? (Use the ZOOM if you need to.)*** 

> <mark>*~ your answer here ~*</mark>

***How should we quantify and monitor for convergent behaviour?*** 

> <mark>*~ your answer here ~*</mark>

***Calculate the true solutions for $y(0.2)$ - how do the estimates above compare?*** 

> <mark>*~ your answer here ~*</mark>

***The [Bulirsch-Stoer](https://en.wikipedia.org/wiki/Bulirsch%E2%80%93Stoer_algorithm) algorithm fits a function to the points on the RIGHTHAND plot and extrapolates the result at $h=0$. What value does this correspond to on the horizontal axis?*** 

> <mark>*~ your answer here ~*</mark>